## Описание задачи

Суть задачи:

Имея входные параметры: размеры помещения, количество маяков, расставить с помощью генетического алгоритма маяки так, чтобы обеспечивался минимум среднего геометрического фактора (ДОП https://ru.wikipedia.org/wiki/Геометрический_фактор). 

Выходные параметры: координаты маяков.

Данная задача решает оптимальную расстановку по данному критерию маяков внутри помещения, чтобы обеспечивалась максимальная точность локальной навигации.

## Подключение библиотек

In [1]:
pip install plotly

In [2]:
import numpy as np
import random as random
from numpy import mean
import matplotlib.path as mpltPath
import matplotlib.pyplot as plt
from matplotlib import path

import time
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline

## Функции внутренние

In [3]:
'''
    Функция получения точек, заполняющих помещение,
        число углов комнаты задается в rooms
        Входные параметры:
        - rooms - координаты комнаты [x, y]
        - kstepx - шаг сетки по оси х
        - kstepy - шаг сетки по оси y
        
'''


def create_points(rooms, kstepx, kstepy):
    # превращаем лист координат комнаты в array
    rooms = np.array(rooms)
    # находим критические точки комнаты
    xmin = min(rooms[:, 0])
    xmax = max(rooms[:, 0])
    ymin = min(rooms[:, 1])
    ymax = max(rooms[:, 1])

    # делаем сетку для анализа комнаты
    kx = (xmax - xmin) / kstepx
    ky = (ymax - ymin) / kstepy
    # делаем сетку по x
    x = np.arange(xmin, xmax, kx)
    # делаем сетку по y
    y = np.arange(ymin, ymax, ky)

    # формируем координаты комнаты
    xx, yy = np.meshgrid(x, y)

    # формируем path комнаты
    path = mpltPath.Path(rooms)

    points = []
    for i in range(len(xx)):
        for j in range(len(xx[i])):
            points.append([xx[i][j], yy[i][j]])

    # определяем какие точки сетки лежат вне path комнаты
    inside2 = path.contains_points(points)

    # делаем пустые списки для точек внутри комнаты,
    # и для внешних точек, которые не попали в комнату
    pointsIn = []
    pointsOut = []

    for i in range(len(inside2)):
        if inside2[i] == True:
            pointsIn.append(points[i])
        else:
            pointsOut.append(points[i])

    pointsIn = np.array(pointsIn)
    pointsOut = np.array(pointsOut)
    # возвращаем точки, которые лежат внутри комнаты и которые лежат вне комнаты
    # возвращаем шаг сетки
    return pointsIn, pointsOut, kx, ky


'''
    Функция создания координат маяков
        Входные параметры:
        - pointsIn - внутренние координаты комнаты
        - nmayak - число маяков 
'''


def create_coord_mayak(pointsIn, nmayak):
    # создаем пустую матрицу под координаты маяков
    coord_mayak = np.zeros((nmayak, 2))
    for i in range(nmayak):
        # случайным образом берем координаты маяков из набора точек внутри помещения
        coord_mayak[i] = random.choice(pointsIn)
    return coord_mayak


'''
    Функция учёта координат маяков внутри помещения
        Входные параметры:
        - coord_mayak - координаты маяков
        - pointsIn - внутренние координаты комнаты
        - nmayak - число маяков 
'''


def create_points_indoors(coord_mayak, pointsIn, nmayak):
    # создаем пустой массив под индексы координат маяков
    index = []
    for i in range(len(pointsIn)):
        for j in range(nmayak):
            if pointsIn[i][0] == coord_mayak[j][0] and pointsIn[i][1] == coord_mayak[j][1]:
                index.append(i)

    # удаляем координаты маяков из набора внутренних точек
    pointsInMetka = np.delete(pointsIn, index, axis=0)
    return pointsInMetka

'''
    Функция для проверки прямой видимости маяка и метки
        Входные параметры:
        - rooms - размеры комнаты
        - mayak - координаты маяка
        - metka - координаты метки
        - ncorner - число углов комнаты
        Выходные параметры:
        - flag - 0 (отрезки не пересекаются); 1 (отрезки пересекаются)
'''

def intersection_segments(rooms, mayak, metka, ncorner):
  
  # объявляем переменную flag
  # flag = 0 - метка и маяк в прямой видимости
  # flag = 1 - метка и маяк не в прямой видимости 
  # (есть пересечение стены и прямой метка-маяк)

  flag_metka = []

  xm1, ym1 = mayak
  xm2, ym2 = metka

  A1 = ym1 - ym2
  B1 = xm2 - xm1
  C1 = xm1 * ym2 - xm2 * ym1

  def intersection_point(x, y):
    if min(xm1, xm2) <= x <= max(xm1, xm2) and min(xc1, xc2) <= x <= max(xc1, xc2) and \
        min(ym1, ym2) <= y <= max(ym1, ym2) and min(yc1, yc2) <= y <= max(yc1, yc2):
      # есть пересечение отрезков
      flag = 1
    else:
      # пересечения отрезков нет
      flag = 0
    return flag

  for i in range(ncorner):

    flag = 0

    xc1, yc1 = rooms[i]
    if i != (ncorner - 1):
      xc2, yc2 = rooms[i+1]
    else:
      xc2, yc2 = rooms[0]

    A2 = yc1 - yc2
    B2 = xc2 - xc1
    C2 = xc1 * yc2 - xc2 * yc1
    
    if (B1*A2 - B2*A1) != 0 and A1 != 0:
      y = (C2*A1 - C1*A2) / (B1*A2 - B2*A1)
      x = (-C1 - B1*y) / A1
      flag = intersection_point(x, y)
    else: 
      flag = 0

    if (B1*A2 - B2*A1) != 0 and A2 != 0:
      y = (C2*A1 - C1*A2) / (B1*A2 - B2*A1)
      x = (-C2 - B2*y) / A2
      flag = intersection_point(x, y)
    else:
      # пересечения отрезков нет, отрезки параллельны
      flag = 0
    
    flag_metka.append(flag)

  return max(flag_metka)

'''
    Функция расчета геометрического фактора для всех точек внутри помещения
        Входные параметры:
        - rooms - координаты комнаты
        - pointsInMetka - координаты оставшихся внутренних точек
        - coord_mayak - координаты маяков
        - nmayak - число маяков 
        - factor - значение DOP для точек, для которых невозможно вычислить DOP
'''

def create_dop_factor(rooms, pointsInMetka, coord_mayak, nmayak, factor):
    # создаем пустую матрицу DOP
    DOP = np.zeros(len(pointsInMetka))
    # заполняем DOP значением, где невозможно произвести расчет DOP
    DOP[:] = factor
    # объявляем переменную - для скольких точек можно вычислить DOP
    # когда метку видят минимум 3 маяка
    points_true = 0


    # делаем цикл по всем точкам внутри помещения (кроме координат маяков)
    for i in range(len(pointsInMetka)):
        # объявляем матрицу расстояний от метки до маяков [rij]
        rast_matrix = np.zeros(nmayak)
        # объявляем градиентную матрицу
        grad_matrix = np.zeros((nmayak, 2))
        # объявляем переменную - число маяков, которые в прямой видимости у взятой i-ой метки
        flag_mayak = 0

        # заходим в цикл для каждого маяка
        # здесь строить точки с прямой и проверять прямые соединяются или нет
        for j in range(nmayak):

          flag = intersection_segments(rooms, coord_mayak[j], pointsInMetka[i], ncorner)

          if flag == 0:
            # увеличиваем переменную - сколько маяков видят метку
            flag_mayak += 1

            # если координата по х метки равна координате по х маяка
            # получаем, что метка и маяк образую собой вертикальную прямую - меняется только значение y
            if (pointsInMetka[i][0] - coord_mayak[j][0]) == 0 and (pointsInMetka[i][1] - coord_mayak[j][1]) != 0:
                # рассчиываем расстояние до каждого маяка
                # из формулы убираем значения для расстояния по х - нулевое, т.к. прямая вертикальна
                rast_matrix[j] = np.sqrt((pointsInMetka[i][1] - coord_mayak[j][1]) ** 2)
                grad_matrix[j] = (pointsInMetka[i] - coord_mayak[j]) / rast_matrix[j]
                

            # если координата по y метки равна координате по y маяка
            # получаем, что метка и маяк образую собой горизонтальную прямую - меняется только значение x

            elif (pointsInMetka[i][1] - coord_mayak[j][1]) == 0 and (pointsInMetka[i][0] - coord_mayak[j][0]) != 0:
                rast_matrix[j] = np.sqrt((pointsInMetka[i][0] - coord_mayak[j][0]) ** 2)
                grad_matrix[j] = (pointsInMetka[i] - coord_mayak[j]) / rast_matrix[j]

            # если точка и маяк лежат на кривой с наклоном
            # здесь также можно рассмотреть следующий цикл - будет строиться прямая и сравнение с каждой прямой,
            # которая образует стенку комнаты
            elif (pointsInMetka[i][1] - coord_mayak[j][1]) != 0 and (pointsInMetka[i][0] - coord_mayak[j][0]) != 0:
                rast_matrix[j] = (np.sqrt((pointsInMetka[i][0] - coord_mayak[j][0]) ** 2 +
                                          (pointsInMetka[i][1] - coord_mayak[j][1]) ** 2))
                grad_matrix[j] = (pointsInMetka[i] - coord_mayak[j]) / rast_matrix[j]

        # если для одной метки насчитали 3 и более маяка в зоне видимости, то можно рассчитать DOP
        if flag_mayak >= 3:
            # увеличиваем число переменной для которой можно высчитать DOP
            points_true += 1

            if np.isfinite(np.linalg.cond(grad_matrix.T)):
                DOP[i] = np.sqrt(np.trace(np.linalg.inv((grad_matrix.T).dot(grad_matrix))))
            else:
                print('singular matrix')
            # DOP[i] = np.sqrt(np.trace(np.linalg.inv((grad_matrix.T).dot(grad_matrix))))

    return DOP, points_true

## Функции отрисовки

In [4]:
def plot_rooms(rooms, pointsIn, pointsOut):
    rooms_plot = rooms.copy()
    rooms_plot.append(rooms_plot[0])
    xr, yr = zip(*rooms_plot)

    xin, yin = pointsIn[:, 0], pointsIn[:, 1]
    xout, yout = pointsOut[:, 0], pointsOut[:, 1]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xr, y=yr,
                             mode='lines',
                             name='path'))
    fig.add_trace(go.Scatter(x=xin, y=yin,
                             mode='markers',
                             name='points in path'))
    fig.add_trace(go.Scatter(x=xout, y=yout,
                             mode='markers', name='points out path'))

    fig.show()


def plot_mayak(coord_mayak, rooms):
    rooms_plot = rooms.copy()
    rooms_plot.append(rooms_plot[0])
    xr, yr = zip(*rooms_plot)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xr, y=yr,
                             mode='lines',
                             name='path'))
    fig.add_trace(go.Scatter(x=coord_mayak[:, 0], y=coord_mayak[:, 1],
                             mode='markers',
                             name='coord mayak'))

    fig.show()


def plot_dop_factor(rooms, coord_mayak, DOP, pointsInMetka):
    rooms_plot = rooms.copy()
    rooms_plot.append(rooms_plot[0])
    xr, yr = zip(*rooms_plot)

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=xr, y=yr, z=np.zeros(len(xr)),
                               mode='lines',
                               name="rooms"))

    fig.add_trace(go.Mesh3d(x=(pointsInMetka[:, 0]),
                            y=(pointsInMetka[:, 1]),
                            z=(-DOP),
                            opacity=0.5,
                            color='red',
                            name='DOP'
                            ))

    fig.add_trace(go.Scatter3d(x=coord_mayak[:, 0],
                               y=coord_mayak[:, 1],
                               z=np.zeros(len(coord_mayak)),
                               mode='markers',
                               name='coord mayak'
                              ))

    fig.show()

## Функции генетика

In [5]:
'''
    Функция получения выжившей популяции
        Входные параметры:
        - popul - наша популяция
        - popul_metka - наша популяция - внутренние точки (оставшиеся внутренние точки)
        - val - текущие значения
        - nsurv - количество выживших
        - reverse - указываем требуемую операцию поиска результата: максимизация или минимизация
'''


def getSurvPopul(popul, popul_metka, val, nsurv, reverse):
    # массив для новой популяции - координаты маяков
    newpopul = []
    # массив для новой популяции - оставшиеся внутренние точки (вычет координат маяков)
    newpopul_metka = []
    # сортируем значения в val в зависимости от параметра reverse - максимизация или минимизация
    sval = sorted(val, reverse=reverse)
    # проходимся по циклу nsurv-раз (в итоге в newpopul запишется nsurv-лучших показателей)
    for i in range(nsurv):
        # получаем индекс i-того элемента sval в исходном массиве val
        index = val.index(sval[i])
        # в новую папуляцию добавляем элемент из текущей популяции с найденным индексом
        newpopul.append(popul[index])
        # в новую популяцию так же добавляем значения оставшихся точек
        newpopul_metka.append(popul_metka[index])
    # возвращаем новую популяцию (из nsurv элементов) и сортированный список val
    return newpopul, newpopul_metka, sval


'''
    Функция получения родителей
        Входные параметры:
        - curr_popul - текущая популяция
        - curr_popul_metka - текущая популяция оставшихся точек
        - nsurv - количество выживших
'''

def getParents(curr_popul, curr_popul_metka, nsurv):
    # случайный индекс первого родителя в диапазоне от 0 до nsurv - 1
    indexp1 = random.randint(0, nsurv - 1)
    # случайный индекс второго родителя в диапазоне от 0 до nsurv - 1
    indexp2 = random.randint(0, nsurv - 1)
    # получаем первого бота-родителя по indexp1
    botp1 = curr_popul[indexp1]
    # получаем второго бота-родителя по indexp2
    botp2 = curr_popul[indexp2]
    # здесь берется бот текущей популяции - координаты четырех маяков - и выбираются остаточные точки
    botp1_metka = curr_popul_metka[indexp1]
    botp2_metka = curr_popul_metka[indexp2]
    return botp1, botp1_metka, botp2, botp2_metka  # Возвращаем обоих полученных ботов


'''
    Функция смешивания (кроссинговера) двух родителей 
        Входные параметры:
        - botp1 - первый бот-родитель
        - botp2 - второй бот-родитель
        - pointsIn - все внутренние точки
        - j - номер компонента бота - координаты одного маяка
'''

def crossPointFrom2Parents(botp1, botp2, pointsIn, j):
    pindex = random.random()  # Получаем случайное число в диапазоне от 0 до 1
    # Если pindex меньше 0.5, то берем значения от первого бота, иначе от второго
    if pindex < 0.5:
        # берем за x координаты одного маяка
        x = botp1[j]
        # в качестве y будет браться следующее - все точки внутри за вычетом данной координаты
        index = 0
        for i in range(len(pointsIn)):
            if pointsIn[i][0] == x[0] and pointsIn[i][1] == x[1]:
                index = i
        y = np.delete(pointsIn, index, axis=0)
    else:
        x = botp2[j]
        index = 0
        for i in range(len(pointsIn)):
            if pointsIn[i][0] == x[0] and pointsIn[i][1] == x[1]:
                index = i
        y = np.delete(pointsIn, index, axis=0)

    # смешивание родителей ботов
    # в итоге получаем координаты одного маяка и оставшиеся точки

    return x, y  # возвращаем значение бота

Начальные данные :

1. Количество маяков
2. Число углов комнаты
3. Координаты комнаты


In [6]:
# число маяков
nmayak = 4
# координаты комнаты
xrooms1, yrooms1 = 0, 0
xrooms2, yrooms2 = 1.15, 1
xrooms3, yrooms3 = 1, 3
xrooms4, yrooms4 = 2, -1
# задаем комнату
rooms = [[xrooms1, yrooms1], [xrooms2, yrooms2],
         [xrooms3, yrooms3], [xrooms4, yrooms4]]
# число углов комнаты
ncorner = len(rooms)
# шаг точек сетки - варьируемый параметр
kstepx, kstepy = 100, 50

In [7]:
# размер популяции
npopul = 20
# количество выживших особей (столько лучших ботов перейдет в новую популяцию)
nsurv = 10
# количество новых ботов (столько новых ботов будет создано)
nnew = npopul - nsurv
# размер бота (nmayak, 2) - число маяков на их координаты
# число эпох алгоритма
epohs = 20
# наказываем за невалидные точки
factor = 20

In [8]:
pointsIn, pointsOut, kx, ky = create_points(rooms, kstepx, kstepy)

In [9]:
coord_mayak = create_coord_mayak(pointsIn, nmayak)

In [10]:
plot_mayak(coord_mayak, rooms)

In [11]:
# трехмерный массив популяции [n,nmayak,2] 
popul = []
# двумерный массив популяции [n, len(pointsInMetka]
popul_metka = []
# одномерный массив значений этих ботов - val(mean(DOP))
val = []

plotmeanval = []
plotminval = []

# рассматриваем расставку мяков при одной и той же конфигурации комнаты,
# тогда одиннаковые pointsOut, pointsIn
pointsIn, pointsOut, kx, ky = create_points(rooms, kstepx, kstepy)

# проходим по всей длине популяции
for i in range(npopul):
    # создаем массив координат маяков
    p = create_coord_mayak(pointsIn, nmayak)
    # создаем массив точек внутри помещения с учетом вычета координат маяков
    pi = create_points_indoors(p, pointsIn, nmayak)
    # добавляем компоненты в бота
    popul.append(p)
    popul_metka.append(pi)

# проходим по всем эпохам
for it in range(epohs):
    # создаем пустой список для значений ботов
    val = []
    
    # проходим по всей длине популяции
    for i in range(npopul):
        # берем очередного бота
        bot = popul[i]
        bot_metka = popul_metka[i]
        # высчитываем DOP для каждого бота
        DOP, points_true = create_dop_factor(rooms, bot_metka, bot, ncorner, factor)
        #plot_dop_factor(rooms, bot, DOP, bot_metka)
        # добавляем среднее значение в список
        val.append(sum(DOP) / len(DOP))

    # получаем новую популяцию
    newpopul, newpopul_metka, sval = getSurvPopul(popul, popul_metka, val, nsurv, 0)
    # выводим 5 лучших ботов
    print('')
    print(it, " ", [round(s, 8) for s in sval[0:5]])

    # добавляем среднее значение в список
    plotmeanval.append(val)
    # добавляем минимальное значение в список
    plotminval.append(sval[0])

    # проходимся по цикло nnew раз
    for i in range(nnew):
        # из newpopul (новой популяции) получаем двух случайных родителей-ботов
        # вытаскиваем координаты маяков и внешние точки
        botp1, botp1_metka, botp2, botp2_metka = getParents(newpopul, newpopul_metka, nsurv)

        # массив для нового бота
        newbot = []
        newbot_metka = []

        # проходимся по длине бота и осуществляем смешивание/скрещивание от родителей
        for j in range(nmayak):
            # получаем координаты одного маяка и остальные точки для этого маяка
            x, y = crossPointFrom2Parents(botp1, botp2, pointsIn, j)
            # координата маяка + все точки внутри помещения, кроме заданной

            # создаем область точек из которых будем выбирать новое значение маяка
            points_area_x = []
            points_area_y = []
            # чтобы смещалось вбок на пару точек

            # проходимся циклом по всем внутренним точкам
            # а если точка самая крайняя - если нет таких x точек, нет таких y точек
            for i in range(len(y)):
                # находим все точки внутри помещения, которые на таком же расстоянии x
                if y[i, 0] == x[0]:
                    # добавляем все y
                    points_area_y.append(y[i, 1])
                # находим все точки внутри помещения, которые на таком же расстоянии y
                if y[i, 1] == x[1]:
                    # добавляем все x
                    points_area_x.append(y[i, 0])
            
            # проверить на пустой список - и если пустой то это та-же точка и оставить её
            # если таких точек нет - значит точка граничная
            if len(points_area_x) == 0:
              points_area_x.append(x[0])
            if len(points_area_y) == 0:
              points_area_y.append(x[1])

            # находим область точек
            xmin = min(points_area_x)
            xmax = max(points_area_x)
            ymin = min(points_area_y)
            ymax = max(points_area_y)

            # формируем какое-то число от 0 до 1
            pindex = random.random()
            # eсли pindex меньше 0.5, то берем значения от первого бота, иначе от второго

            index_x = []
            index_y = []

            for i in range(len(y)):
                if y[i, 0] >= xmin and y[i, 0] <= xmax:
                    index_x.append(i)
                if y[i, 1] >= ymin and y[i, 1] <= ymax:
                    index_y.append(i)

            if len(index_x) == 0:
                iy = index_y[random.randint(0, len(index_y) - 1)]
                x = y[iy]
            if len(index_y) == 0:
                ix = index_x[random.randint(0, len(index_x) - 1)]
                x = y[ix]
            if len(index_x) != 0 and len(index_y) !=0:
                if pindex < 0.5:
                    iy = index_y[random.randint(0, len(index_y) - 1)]
                    x = y[iy]
                else:
                    ix = index_x[random.randint(0, len(index_x) - 1)]
                    x = y[ix]

            # тут могут улетать точки за пределы ибо это надо одновременно значит можно менять что-то одно таща за собой вторую координату
            newbot.append(list(x))

        newpopul_metka.append(create_points_indoors(newbot, pointsIn, ncorner))
        newpopul.append(np.array(newbot))

    popul = newpopul
    popul_metka = newpopul_metka


0   [1.81812635, 2.41638444, 2.77475178, 3.00531829, 3.18376005]

1   [1.81812635, 2.41638444, 2.47499062, 2.77475178, 3.00531829]

2   [1.81812635, 1.98801497, 2.00527753, 2.41638444, 2.47499062]

3   [1.81812635, 1.98801497, 2.00527753, 2.35535819, 2.41638444]

4   [1.78528797, 1.81812635, 1.98801497, 2.00527753, 2.35535819]

5   [1.78528797, 1.81812635, 1.98801497, 1.9915506, 2.00527753]

6   [1.78528797, 1.81812635, 1.98801497, 1.9915506, 2.00527753]
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular matrix
singular mat

Singular matrix

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, it+1)), y=plotminval,
                    mode='lines+markers',
                    name='lines'))
fig.update_layout(title='Chart min val DOP',
                   xaxis_title='epohs',
                   yaxis_title='min val DOP')
fig.show()

In [20]:
# выделим лучшего бота
bot = popul[1]
bot_metka = popul_metka[1]

In [21]:
plot_mayak(bot, rooms)

In [22]:
bot

array([[ 1.26, -0.12],
       [ 0.44,  0.36],
       [ 1.08,  1.96],
       [ 1.64,  0.2 ]])

In [27]:
DOP, points_true = create_dop_factor(rooms, bot_metka, bot, ncorner, factor=10)

In [28]:
plot_dop_factor(rooms, bot, DOP, bot_metka)

In [25]:
plot_rooms(rooms, pointsIn, pointsOut)